## GRAPH SEARCH AND CONNECTIVITY 

### Graph representation

In [31]:
# adjacency list representation
G = {
    'a' : ['b','c'],
    'b' : ['a','c'],
    'c' : ['a','d'],
    'd' : []
}

### Search - BFS

In [32]:
from collections import deque

In [81]:
def bfs(G, s):
    """Does BFS search on the given graph

    Args:
      G: graph represented in adjacency list representation
      s: source node
    Raises:
        None
    Returns:
        List of nodes explored in bfs order
    """
    explored = set()
    q = deque()
    res = []
    explored.add(s)
    q.appendleft(s)
    while q: #repeat until queue is not empty
        u = q.pop()
        res.append(u)
        for v in G[u]: #iterate through all the edges
            if v not in explored:
                q.appendleft(v)
                explored.add(v)
    return res

In [95]:
G = {
    's' : ['a','b'],
    'a' : ['b','c'],
    'b' : ['s','d'],
    'c' : ['a','e','d'],
    'd' : ['c','b','e'],
    'e' : ['c','d']
}
bfs(G,'s')

['s', 'a', 'b', 'c', 'd', 'e']

### BFS shortest path unweighted graph

In [54]:
def shortest_path(G,s):
    """Does BFS search on the given graph to find the shortest distance
    from given source

    Args:
      G: graph represented in adjacency list representation
      s: source node
    Raises:
        None
    Returns:
        List of (nodes,shortest_distance) tuples
    """
    explored = set()
    q = deque()
    res = []
    explored.add(s)
    q.appendleft((s,0)) #set source distance to 0
    while q: #repeat until queue is not empty
        u,dist = q.pop()
        res.append((u,dist))
        for v in G[u]: #iterate through all the edges
            if v not in explored:
                q.appendleft((v,dist+1))
                explored.add(v)
    return res

In [96]:
G = {
    's' : ['a','b'],
    'a' : ['b','c'],
    'b' : ['s','d'],
    'c' : ['a','e','d'],
    'd' : ['c','b','e'],
    'e' : ['c','d']
}
shortest_path(G,'s')

[('s', 0), ('a', 1), ('b', 1), ('c', 2), ('d', 2), ('e', 3)]

### Connected components

In [79]:
def connected_components(G):
    """Finds connected components in an undirected graph G

    Args:
      G: graph represented in adjacency list representation
    Raises:
        None
    Returns:
        List of List of connected Components
    """
    explored = set()
    res = []
    
    for v in G:
        if v not in explored:
            nodes = bfs(G,v)
            explored.update(nodes)
            res.append(nodes)
    return res

In [80]:
G = {
    1: [3,5],
    2: [4],
    3: [1,5],
    4: [2],
    5: [1,3,7,9],
    6: [8,10],
    7: [5],
    8: [6,10],
    9: [5],
    10:[6,8]
}

connected_components(G)

[[1, 3, 5, 7, 9], [2, 4], [6, 8, 10]]

### Search - DFS

In [93]:
def dfs_util(G,n,explored,res):
    explored.add(n)
    res.append(n)
    for v in G[n]:
        if v not in explored:
            dfs_util(G,v,explored,res)

def dfs(G,s):
    """Does DFS search on the given graph

    Args:
      G: graph represented in adjacency list representation
      s: source node
    Raises:
        None
    Returns:
        List of nodes explored in dfs order
    """
    explored = set()
    res = []
    dfs_util(G,s,explored,res)
    return res

In [94]:
G = {
    's' : ['a','b'],
    'a' : ['b','c'],
    'b' : ['s','d'],
    'c' : ['a','e','d'],
    'd' : ['c','b','e'],
    'e' : ['c','d']
}
dfs(G,'s')

['s', 'a', 'b', 'd', 'c', 'e']

### Topological sort

In [116]:
def topological_sort(G):
    """Does topological sort on the DAG 

    Args:
      G: DAG represented in adjacency list
    Raises:
        None
    Returns:
        topologically sorted list
    """
    explored = set()
    res = []
    for v in G:
        if v not in explored:
            tmp = []
            dfs_util(G,v,explored,tmp)
            explored.update(tmp)
            res.extend(tmp)
    return res

In [117]:
G = {
    's' : ['v','w'],
    'v' : ['t'],
    'w' : ['t'],
    't' : [],
    'a' : ['b','c'],
    'b' : ['s'],
    'c' : []
}
topological_sort(G)

['s', 'v', 't', 'w', 'a', 'b', 'c']

### Strongly connected components

In [296]:
import logging
# module to reverse graph
def reverse_graph(d):
    reversed_dict = {}
    for key in d:
        reversed_dict[key] = list()
    
    for key, values in d.items():
        for value in values:
            reversed_dict[value].append(key)
    return reversed_dict

In [178]:
# helper function for scc
def dfs_loop(G,n,explored,res,order,ft):
    explored.add(n)
    res.append(n)
    for v in G[n]:
        if v not in explored:
            dfs_loop(G,v,explored,res,order,ft)        
    order[ft[0]] = n
    ft[0] = ft[0]+1

In [298]:
def scc(G,reverse=None):
    """Computes strongly connected components of given grapg G  

    Args:
      G: Graph represented in adjacency list
      reverse: reverse function og graph G
    Raises:
        None
    Returns:
        list of list of SCC
    """
    order = [None]*len(G)
    ft = [0]
    explored = set()
    _G = reverse(G) if reverse else reverse_graph(G)
    logging.debug("dfs on reversed graph")
    #run DFS on reversed graph
    for v in _G:
        if v not in explored:
            dfs_loop(_G,v,explored,[],order,ft)
    #run DFS on original graph in order of finish time
    logging.debug("dfs on original graph")
    explored = set()
    ft = [0]
    res = []
    for v in reversed(order):
        if v not in explored:
            lst = []
            dfs_loop(G,v,explored,lst,[None]*len(G),ft)
            res.append(lst)
    return res

In [299]:
G = {
    0: [2,3],
    1: [0],
    2: [1],
    3: [4],
    4: []
}

scc(G)

[[4], [3], [0, 2, 1]]

### assignment

In [300]:
import urllib


In [319]:
#url = 'https://raw.githubusercontent.com/beaunus/stanford-algs/master/testCases/course2/assignment1SCC/input_mostlyCycles_10_32.txt'
url = 'https://raw.githubusercontent.com/beaunus/stanford-algs/master/testCases/course2/assignment1SCC/input_mostlyCycles_17_128.txt'
from urllib.request import urlopen
dat = urlopen(url).readlines()


In [320]:
print(len(dat))

136


In [322]:
# initialize Graph and reverse graph
G = {}
_G = {}
for i in range(1,128+1):
    G[str(i)] = []
    _G[str(i)] = []

In [323]:
#populate graph and reverse graph
def clean(l):
    return str(l,'utf-8').strip().split()
    
for l in dat:
    (v,w) = clean(l)
    G[v].append(w)
    _G[w].append(v)
    

In [324]:
def my_reversgraph(G):
    global _G
    return _G

In [325]:

res = scc(G)

In [326]:
#get top 5
import heapq
heapq.nlargest(5,[len(r) for r in res])


[119, 9]